In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import sys
import os
import pandas as pd
import numpy as np
import astropy.units as u
from datetime import date
from astroquery.simbad import Simbad
from astroquery.vizier import Vizier
from astroquery import nasa_ads as na

In [ ]:
def unique(list_of_values):
    unique_values = []
    for _, value in enumerate(list_of_values):
        if value in unique_values:
            pass
        else:
            unique_values.append(value)
    return unique_values

In [ ]:
primordial_disks_directory = '/Users/lib278/Documents/Manuscripts/PRIMORDIAL_DISKS/'
primordial_disks_sample_directory = primordial_disks_directory+'Tables/Condensed_Sample_By_Date/'

today = date.today()
date_str = today.strftime("%Y-%b-%d")
date_str

sheet_url = "https://docs.google.com/spreadsheets/d/1xuMXHVTw3_iwn3tgzBoIaXaMP81ULbpIURthZyvs7iw/edit#gid=0"
url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
raw_df = pd.read_csv(url_1)

targets_for_query = unique(raw_df['Target'].values)
print(len(targets_for_query))
targ_list = unique(raw_df['Target'].values)

367


In [ ]:
customSimbad = Simbad()
customSimbad.add_votable_fields('ra(2;A;ICRS;J2017.5;2000)', 'dec(2;D;ICRS;J2017.5;2000)')
customSimbad.remove_votable_fields('coordinates')
customSimbad.TIMEOUT = 4 * 60 # sets the timeout to 4 min
customSimbad.ROW_LIMIT = -1

# v = Vizier(columns=['_RAJ2000', '_DEJ2000','B-V', 'Vmag', 'Plx'],
#            column_filters={"Vmag":">10"}, keywords=["optical", "xry"])

custom_vizier = Vizier(columns=['*'])
custom_vizier.TIMEOUT = 10 * 60 # sets the timeout to 4 min
custom_vizier.ROW_LIMIT = -1
custom_vizier.COLUMN_LIMIT = -1
custom_vizier.CATALOGUE_LIMIT = -1

In [5]:
target = 'AA Tau'
alternate_IDs = customSimbad.query_objectids(target)
Simbad_search = customSimbad.query_object(target)
main_ID = Simbad_search[Simbad_search.keys()[0]][0]
RA = Simbad_search[Simbad_search.keys()[1]][0]
DEC = Simbad_search[Simbad_search.keys()[2]][0]
print(main_ID, '|', RA, '|', DEC)

In [6]:
# result = custom_vizier.query_object(target)
# tables = result.keys()
# str(result[tables[272]].columns['vsini'].unit)

In [7]:
# cat_split = [part for x in result[tables[-1]].meta['name'].split('/') for part in (x, '/')]
# cat_name = cat_split[0]
# for part_i in range(1, len(cat_split)-3):
#     cat_name += cat_split[part_i]
# # cat = custom_vizier.find_catalogs(cat_name)
# cat = custom_vizier.find_catalogs(cat_name, max_catalogs=1e6)

# # result.format_table_list()  # .split(',')
# # result.values()[272].meta['name']
# # result.print_table_list()
# cat_dict = ({k: v.description for k, v in cat.items()})
# cat_title = cat_dict[cat_name].split(' (')[0]
# cat_author = cat_dict[cat_name].split(' (')[1].split('+')[0]
# cat_year = cat_dict[cat_name].split(' (')[1].split(', ')[1][0:-1]
# cat_year

In [8]:
# na.ADS.TOKEN = 'kRJ0yK4fcG45mR6a6A27vN2kosc8KnNfZ3zH7THa'


# ads_search_string = 'J/A+A/378/116/'
# ads_search_string = '^' + cat_author

# ads_search_string = "author:'^" + cat_author + "'"  #  abs:'" + cat_title + "'"   # "' year:'" + cat_year + "'"   
# ads_search_string = "title:'" + cat_title + "'" 
# print(ads_search_string)

# # # na.ADS.ADS_FIELDS = 
# ADS_query = na.ADS.query_simple(ads_search_string)
# ADS_query['pubdate']

In [9]:
Rstar_chars = ['Rad', 'rad', 'Radius', 'radius', 'R*', 'r*', 'Rstar', 'rstar', 'R_star', 'r_star', 'R', 'r']
vsini_chars = ['Vsini','vsini','v_sini','V_sini','vsin_i','Vsin_i','v_sin_i','V_sin_i','vsin(i)','Vsin(i)','v_sin(i)','V_sin(i)']

In [10]:
nodat_targs = {}
nodat_targs = {'vsini': [],
               'Rstar': [],
               'Prot': [],
               }

In [12]:
query_par = 'Rstar
save_dir = primordial_disks_directory + 'Tables/astroquery/' + query_par + '/'

skip_targs = []
for file in os.listdir(save_dir):
    skip_targs.append(file.split('.')[0])
print(skip_targs)

unresolved_targs = []


for targ in targ_list:
    Simbad_search = customSimbad.query_object(targ)
    try:
        target = Simbad_search[Simbad_search.keys()[0]][0]
    except:
        unresolved_targs.append(targ)
        print('\nUnresolved: ' + str(unresolved_targs))
        continue

    RA = Simbad_search[Simbad_search.keys()[1]][0]
    DEC = Simbad_search[Simbad_search.keys()[2]][0]

    if target in skip_targs:
        continue
    if target in nodat_targs[query_par]:
        continue

    alternate_IDs = customSimbad.query_objectids(targ)['ID']
    for alt_ID_i, alt_ID in enumerate(alternate_IDs):
        if alt_ID != target:
            if 'alt_ID_str' not in locals():
                alt_ID_str = alt_ID + ','
            else:
                alt_ID_str += alt_ID + ','
        if '2MASS' in alt_ID:
            twoMASS = alt_ID
            
    print(' ')
    print(target, '|', RA, '|', DEC)

    result = custom_vizier.query_object(target)
    tables = result.keys()

    cat_data = {}
    cat_data = {'target': [],
                'RA': [],
                'DEC': [],
                'col_chars': [],
                'columns': [],
                'units': [],
                'values': [],
                'descriptions': [],
                'catalogues': [],
                'titles': [],
                'meta': [],
                '2MASS': [],
                'altIDs': [],
                }
    for tab in range(len(tables)):
        meta = []
        columns = []
        values = []
        units = []
        descriptions = []
        catalogues = []
        titles = []
        for bep in range(len(result[tables[tab]].columns)):
            column = str(result[tables[tab]].columns[bep].name)
            unit = str(result[tables[tab]].columns[bep].unit)
            meta = result[tables[tab]].meta

            for col_char in vsini_chars:
                if ('km / s' in unit) and (col_char in column):
                    cat_split = [part for x in result[tables[tab]].meta['name'].split('/') for part in (x, '/')]
                    cat_name = cat_split[0]
                    for part_i in range(1, len(cat_split)-3):
                        cat_name += cat_split[part_i]
                    try:
                        cat = custom_vizier.find_catalogs(keywords=cat_name, max_catalogs=1e6)
                    except:
                        continue
                    cat_dict = ({k: v.description for k, v in cat.items()})
                    cat_data['target'].append(target)
                    cat_data['RA'].append(RA)
                    cat_data['DEC'].append(DEC)
                    cat_data['columns'].append(result[tables[tab]].columns[bep].name)
                    cat_data['col_chars'].append(col_char)
                    cat_data['units'].append(result[tables[tab]].columns[bep].unit)
                    cat_data['values'].append(result[tables[tab]].columns[bep].value.data[-1])
                    cat_data['descriptions'].append(result[tables[tab]].columns[bep].description)
                    cat_data['catalogues'].append(result.values()[tab].meta['name'])
                    cat_data['titles'].append(cat_dict[cat_name])
                    cat_data['meta'].append(result[tables[tab]].meta['description'])
                    cat_data['2MASS'].append(twoMASS)
                    cat_data['altIDs'].append(alt_ID_str)

    if 'alt_ID_str' in locals():
        del alt_ID_str
        
    cat_df = pd.DataFrame(cat_data)
    if len(cat_df) > 0:
        cat_df.to_csv(save_dir + target + '.csv', index=False)
    else:
        nodat_targs[query_par].append(target)
        print('\nNo dat: ' + str(nodat_targs[query_par]))
        
    print('----------------------------------------------------------------')


['V* CV Cha', 'V* CI Tau', 'V* AB Aur', '', 'NAME CoKu Tau 1', '2MASS J03285129+3117397', 'V* CW Tau', 'Hen 3-545', 'Haro 5-64', 'V* AK Sco', 'V* AA Tau', 'CRBR 2422', 'V* CQ Tau', 'V* CX Tau', 'IRAS 11051-7706', 'V* CY Tau', '[CCE98] 1-46', 'V* BP Tau', 'V* CS Cha']

Unresolved: AS 205B

Unresolved: IRAS 19347+0727
 
CRBR 2422.8-3423 | 16 27 24.5975 | -24 41 03.772


/Users/lib278/opt/anaconda3/envs/pyKLIP/lib/python3.9/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : EM* AS  205B
  warnings.warn("Warning: The script line number %i raised "
/Users/lib278/opt/anaconda3/envs/pyKLIP/lib/python3.9/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 3 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : EM* AS  205B
  warnings.warn("Warning: The script line number %i raised "
/Users/lib278/opt/anaconda3/envs/pyKLIP/lib/python3.9/site-packages/astroquery/simbad/core.py:135: UserWarning: Warning: The script line number 2 raised an error (recorded in the `errors` attribute of the result table): Identifier not found in the database : IRAS 19347+0727
  warnings.warn("Warning: The script line number %i raised 

----------------------------------------------------------------

Unresolved: DD Tau A
 
V* DE Tau | 04 21 55.6488 | +27 55 05.708


/Users/lib278/opt/anaconda3/envs/pyKLIP/lib/python3.9/site-packages/astroquery/vizier/core.py:769: UserWarning: VOTABLE parsing raised exception: 46084:4: not well-formed (invalid token)
  warnings.warn("VOTABLE parsing raised exception: {0}".format(ex))


----------------------------------------------------------------
 
V* DF Tau | 04 27 02.7960 | +25 42 21.990


----------------------------------------------------------------
 
V* DG Tau | 04 27 04.6992 | +26 06 15.701


/Users/lib278/opt/anaconda3/envs/pyKLIP/lib/python3.9/site-packages/astroquery/vizier/core.py:769: UserWarning: VOTABLE parsing raised exception: 46084:4: not well-formed (invalid token)
  warnings.warn("VOTABLE parsing raised exception: {0}".format(ex))
